In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

2022-09-27 19:16:44.515107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-27 19:16:44.515141: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
db_path = "./mnli.db"
ds_name = 'mnli'
wandb_project = 'mnli_search'
wandb_entity = 'johntzwei'
seed = 0
annealing_runs = 2000
n_downsample = 2000
n_search = 200

model_card = "google/electra-small-discriminator"
pretraining = True
max_steps = 10000
eval_steps = 300
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [4]:
training_args = select.SubsetTrainingArguments(model_card=model_card,
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)
searching_args = select.SubsetSearcherArguments(seed=seed, 
                                                db_path=db_path,
                                                wandb_project=wandb_project,
                                                wandb_entity=wandb_entity
                                               )

In [5]:
processed_ds = preprocess.get_dataset(ds_name, training_args.model_card)

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-5e912404f35e1845.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-a8efe87a750447a6.arrow


  0%|          | 0/2210 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-abedee5dbca0c22b.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-0e96b66df4e50237.arrow


  0%|          | 0/2210 [00:00<?, ?ex/s]

In [6]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds["validation"], test_ds=processed_ds["validation"]
)

In [7]:
data_pool = processed_ds["train"].shuffle(seed=searching_args.seed).select(range(searching_args.data_pool_size))

Loading cached shuffled indices for dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-fdaab7755b49ddcd.arrow


In [8]:
subset_searcher = select.SubsetSearcher(subset_trainer=subset_trainer, params=searching_args, data_pool=data_pool)

In [ ]:
subset_searcher.search_til_manual_termination()

wandb: Currently logged in as: johntzwei (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-09-27 19:16:57.819372: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-27 19:16:57.819418: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [ ]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(DB_PATH); df